In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
    ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
    ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4437


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(14, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0045
Function value obtained: 0.4714
Current minimum: 0.4714
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0027
Function value obtained: 0.4716
Current minimum: 0.4714
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0026
Function value obtained: 0.4719
Current minimum: 0.4714
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0026
Function value obtained: 0.4717
Current minimum: 0.4714
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.9684
Function value obtained: 0.4716
Current minimum: 0.4709
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 1.0074
Function value obtained: 0.4720
Current minimum: 0.4709
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.9411
Function value obtained: 0.4708
Current minimum: 0.4708
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.9503
Function value obtained: 0.4720
Current minimum: 0.4708
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.7145
Function value obtained: 0.4708
Current minimum: 0.4708
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.1186
Function value obtained: 0.4706
Current minimum: 0.4706
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.1444
Function value obtained: 0.4722
Current minimum: 0.4706
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.1264
Function value obtained: 0.4723
Current minimum: 0.4706
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.1423
Function value obtained: 0.4720
Current minimum: 0.4706
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.1989
Function value obtained: 0.4721
Current minimum: 0.4706
Iteration No: 86 started. Sea

Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.4266
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 21 started. Searching for the next optimal point.
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.4153
Function value obtained: 0.4372
Current minimum: 0.4367
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.4647
Function value obtained: 0.4370
Current minimum: 0.4367
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.4546
Function value obtained: 0.4373
Current minimum: 0.4367
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.4475
Function value obtained: 0.4374
Current minimum: 0.4367
Iteration No: 25 started. Sea

Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 1.3442
Function value obtained: 0.4373
Current minimum: 0.4367
Iteration No: 61 started. Searching for the next optimal point.
Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 1.2162
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 1.2412
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 1.1810
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 1.4134
Function value obtained: 0.4374
Current minimum: 0.4367
Iteration No: 65 started. Sea

...

Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 0.9080
Function value obtained: 0.4625
Current minimum: 0.4620
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 0.7412
Function value obtained: 0.4644
Current minimum: 0.4620
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 1.0614
Function value obtained: 0.4635
Current minimum: 0.4620
Iteration No: 40 started. Searching for the next optimal point.
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.8347
Function value obtained: 0.4635
Current minimum: 0.4620
Iteration No: 41 started. Searching for the next optimal point.
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.9264
Function value obtained: 0.4637
Current minimum: 0.4620
Iteration No: 42 started. Sea

Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 0.9673
Function value obtained: 0.4618
Current minimum: 0.4618
Iteration No: 78 started. Searching for the next optimal point.
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 0.9933
Function value obtained: 0.4638
Current minimum: 0.4618
Iteration No: 79 started. Searching for the next optimal point.
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 0.9361
Function value obtained: 0.4618
Current minimum: 0.4618
Iteration No: 80 started. Searching for the next optimal point.
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 0.9879
Function value obtained: 0.4634
Current minimum: 0.4618
Iteration No: 81 started. Searching for the next optimal point.
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.9881
Function value obtained: 0.4618
Current minimum: 0.4618
Iteration No: 82 started. Sea

Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 0.5174
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 0.4038
Function value obtained: 0.4376
Current minimum: 0.4369
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 0.4613
Function value obtained: 0.4375
Current minimum: 0.4369
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 0.4820
Function value obtained: 0.4371
Current minimum: 0.4369
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.4759
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 21 started. Sea

Iteration No: 56 ended. Search finished for the next optimal point.
Time taken: 1.2514
Function value obtained: 0.4376
Current minimum: 0.4366
Iteration No: 57 started. Searching for the next optimal point.
Iteration No: 57 ended. Search finished for the next optimal point.
Time taken: 1.0819
Function value obtained: 0.4373
Current minimum: 0.4366
Iteration No: 58 started. Searching for the next optimal point.
Iteration No: 58 ended. Search finished for the next optimal point.
Time taken: 1.1917
Function value obtained: 0.4365
Current minimum: 0.4365
Iteration No: 59 started. Searching for the next optimal point.
Iteration No: 59 ended. Search finished for the next optimal point.
Time taken: 1.1329
Function value obtained: 0.4373
Current minimum: 0.4365
Iteration No: 60 started. Searching for the next optimal point.
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 1.0759
Function value obtained: 0.4372
Current minimum: 0.4365
Iteration No: 61 started. Sea

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 1.8024
Function value obtained: 0.4365
Current minimum: 0.4365
Iteration No: 97 started. Searching for the next optimal point.
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 1.6343
Function value obtained: 0.4365
Current minimum: 0.4365
Iteration No: 98 started. Searching for the next optimal point.
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.5893
Function value obtained: 0.4377
Current minimum: 0.4365
Iteration No: 99 started. Searching for the next optimal point.
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 1.5747
Function value obtained: 0.4371
Current minimum: 0.4365
Iteration No: 100 started. Searching for the next optimal point.
Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 1.7418
Function value obtained: 0.4364
Current minimum: 0.4364
[0.023404   0.01373028 0.01

In [10]:
best_weights

{'cohesion': array([0.04058476, 0.03903616, 0.01530053, 0.01530053, 0.01530053,
        0.12517942, 0.01530053, 0.01530053, 0.09137542, 0.15300527,
        0.15300527, 0.15300527, 0.01530053, 0.15300527]),
 'syntax': array([0.10206318, 0.01481414, 0.03711879, 0.13615618, 0.01481414,
        0.14814137, 0.04062218, 0.14814137, 0.11725622, 0.01481414,
        0.03896083, 0.12423626, 0.04804708, 0.01481414]),
 'vocabulary': array([0.16666999, 0.016667  , 0.016667  , 0.16666999, 0.05213163,
        0.07162489, 0.016667  , 0.16666999, 0.0798139 , 0.016667  ,
        0.016667  , 0.16666999, 0.016667  , 0.02974763]),
 'phraseology': array([0.12889642, 0.01288964, 0.01288964, 0.12889642, 0.01288964,
        0.12889642, 0.01288964, 0.08026439, 0.12889642, 0.06901925,
        0.12889642, 0.12889642, 0.01288964, 0.01288964]),
 'grammar': array([0.02918285, 0.01593293, 0.01593293, 0.15932929, 0.01593293,
        0.01593293, 0.04845643, 0.01593293, 0.0543845 , 0.01593293,
        0.15932929, 0.1593

In [11]:
scores

{'cohesion': 0.4705329638140219,
 'syntax': 0.43657194946822636,
 'vocabulary': 0.4043707250949422,
 'phraseology': 0.4458097453712893,
 'grammar': 0.4617774818937242,
 'conventions': 0.43644946561650166}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4426


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.946397,0.915109,0.912567,0.826574,0.887984
syntax,0.946397,1.000000,0.938533,0.961609,0.917591,0.929432
vocabulary,0.915109,0.938533,1.000000,0.960206,0.876171,0.892467
phraseology,0.912567,0.961609,0.960206,1.000000,0.943363,0.882903
grammar,0.826574,0.917591,0.876171,0.943363,1.000000,0.856574
conventions,0.887984,0.929432,0.892467,0.882903,0.856574,1.000000
